# Bayesian Trading Strategy with Backtest

## Week 20: Bayesian Methods in Quantitative Finance

This notebook implements a complete Bayesian trading strategy framework including:
- **Bayesian parameter estimation** for return distributions
- **Online posterior updates** using conjugate priors
- **Probabilistic trading signals** based on posterior beliefs
- **Full backtesting framework** with transaction costs
- **Performance analysis** with key metrics

### Key Concepts:
- **Prior Distribution**: Our initial beliefs about parameters before seeing data
- **Likelihood**: Probability of observed data given parameters
- **Posterior Distribution**: Updated beliefs after observing data via Bayes' theorem:

$$P(\theta|D) = \frac{P(D|\theta) \cdot P(\theta)}{P(D)}$$

## 1. Import Required Libraries

In [3]:
# Core libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Statistical libraries for Bayesian inference
from scipy import stats
from scipy.special import gammaln

# Data fetching
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load and Prepare Market Data

We'll fetch historical price data and compute returns along with technical features that will inform our Bayesian model.

In [4]:
# Define parameters
TICKER = "SPY"  # S&P 500 ETF
START_DATE = "2018-01-01"
END_DATE = "2024-12-31"

# Fetch data (using auto_adjust=True for adjusted prices in 'Close' column)
print(f"Fetching {TICKER} data from {START_DATE} to {END_DATE}...")
data = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)

# Calculate returns (using 'Close' which is now adjusted with auto_adjust=True)
data['Returns'] = data['Close'].pct_change()
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))

# Calculate technical features
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['Volatility_20'] = data['Returns'].rolling(window=20).std() * np.sqrt(252)
data['Momentum_10'] = data['Close'].pct_change(periods=10)

# Calculate rolling mean and std of returns (for feature engineering)
data['Rolling_Mean_20'] = data['Returns'].rolling(window=20).mean()
data['Rolling_Std_20'] = data['Returns'].rolling(window=20).std()

# Drop NaN values
data = data.dropna()

print(f"\nData shape: {data.shape}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"\nSample statistics:")
print(f"  Mean daily return: {data['Returns'].mean()*100:.4f}%")
print(f"  Std daily return: {data['Returns'].std()*100:.4f}%")
print(f"  Annualized return: {data['Returns'].mean()*252*100:.2f}%")
print(f"  Annualized volatility: {data['Returns'].std()*np.sqrt(252)*100:.2f}%")

data.head()

Fetching SPY data from 2018-01-01 to 2024-12-31...


KeyError: 'Adj Close'

## 3. Define Bayesian Model for Price Prediction

### Normal-Inverse-Gamma Conjugate Prior

For modeling returns with unknown mean $\mu$ and variance $\sigma^2$, we use the **Normal-Inverse-Gamma** conjugate prior:

$$\mu | \sigma^2 \sim \mathcal{N}(\mu_0, \sigma^2 / \kappa_0)$$
$$\sigma^2 \sim \text{Inv-Gamma}(\alpha_0, \beta_0)$$

Where:
- $\mu_0$: Prior mean of returns
- $\kappa_0$: Strength of belief in prior mean
- $\alpha_0, \beta_0$: Shape and scale of inverse-gamma for variance

In [ ]:
class BayesianReturnEstimator:
    """
    Bayesian estimator for return distribution using Normal-Inverse-Gamma conjugate prior.
    
    This allows for online updates as new data arrives, maintaining a posterior
    distribution over the mean and variance of returns.
    """
    
    def __init__(self, mu_0=0.0, kappa_0=1.0, alpha_0=2.0, beta_0=0.0001):
        """
        Initialize prior parameters.
        
        Parameters:
        -----------
        mu_0 : float
            Prior mean (default: 0, uninformative about direction)
        kappa_0 : float
            Strength of prior on mean (lower = weaker prior)
        alpha_0 : float
            Shape parameter for inverse-gamma (must be > 1 for finite mean)
        beta_0 : float
            Scale parameter for inverse-gamma
        """
        # Prior hyperparameters
        self.mu_0 = mu_0
        self.kappa_0 = kappa_0
        self.alpha_0 = alpha_0
        self.beta_0 = beta_0
        
        # Current posterior parameters (start at prior)
        self.mu_n = mu_0
        self.kappa_n = kappa_0
        self.alpha_n = alpha_0
        self.beta_n = beta_0
        self.n = 0
        
    def update(self, returns):
        """
        Update posterior with new observations using conjugate update rules.
        
        Parameters:
        -----------
        returns : array-like
            New return observations
        """
        returns = np.atleast_1d(returns)
        n_new = len(returns)
        
        if n_new == 0:
            return
            
        # Sample statistics
        x_bar = np.mean(returns)
        
        # Update posterior parameters
        kappa_new = self.kappa_n + n_new
        mu_new = (self.kappa_n * self.mu_n + n_new * x_bar) / kappa_new
        alpha_new = self.alpha_n + n_new / 2
        
        # Sum of squared deviations
        ss = np.sum((returns - x_bar) ** 2)
        beta_new = self.beta_n + 0.5 * ss + \
                   (self.kappa_n * n_new * (x_bar - self.mu_n) ** 2) / (2 * kappa_new)
        
        # Update state
        self.kappa_n = kappa_new
        self.mu_n = mu_new
        self.alpha_n = alpha_new
        self.beta_n = beta_new
        self.n += n_new
        
    def reset(self):
        """Reset to prior."""
        self.mu_n = self.mu_0
        self.kappa_n = self.kappa_0
        self.alpha_n = self.alpha_0
        self.beta_n = self.beta_0
        self.n = 0
        
    def get_posterior_mean(self):
        """Get posterior mean of returns."""
        return self.mu_n
    
    def get_posterior_variance(self):
        """Get posterior mean of variance (E[sigma^2])."""
        if self.alpha_n > 1:
            return self.beta_n / (self.alpha_n - 1)
        return np.inf
    
    def get_posterior_std(self):
        """Get posterior estimate of standard deviation."""
        return np.sqrt(self.get_posterior_variance())
    
    def get_predictive_distribution(self):
        """
        Get parameters of the posterior predictive distribution.
        The predictive distribution for a new observation is Student-t.
        
        Returns:
        --------
        tuple: (df, loc, scale) for Student-t distribution
        """
        df = 2 * self.alpha_n
        loc = self.mu_n
        scale = np.sqrt(self.beta_n * (self.kappa_n + 1) / (self.alpha_n * self.kappa_n))
        return df, loc, scale
    
    def prob_positive_return(self):
        """
        Calculate P(next return > 0) using posterior predictive.
        
        Returns:
        --------
        float: Probability that next return will be positive
        """
        df, loc, scale = self.get_predictive_distribution()
        # P(X > 0) = 1 - CDF(0)
        return 1 - stats.t.cdf(0, df=df, loc=loc, scale=scale)
    
    def prob_return_above_threshold(self, threshold):
        """
        Calculate P(next return > threshold).
        
        Parameters:
        -----------
        threshold : float
            Return threshold
            
        Returns:
        --------
        float: Probability that next return exceeds threshold
        """
        df, loc, scale = self.get_predictive_distribution()
        return 1 - stats.t.cdf(threshold, df=df, loc=loc, scale=scale)
    
    def expected_return(self):
        """Get expected value of next return from predictive distribution."""
        return self.mu_n
    
    def var_at_risk(self, alpha=0.05):
        """
        Calculate Value at Risk at given confidence level.
        
        Parameters:
        -----------
        alpha : float
            Significance level (default 5%)
            
        Returns:
        --------
        float: VaR (negative number representing loss)
        """
        df, loc, scale = self.get_predictive_distribution()
        return stats.t.ppf(alpha, df=df, loc=loc, scale=scale)


# Test the Bayesian estimator
print("Testing BayesianReturnEstimator...")
estimator = BayesianReturnEstimator()

# Update with first 100 days of returns
test_returns = data['Returns'].values[:100]
estimator.update(test_returns)

print(f"\nAfter observing {estimator.n} returns:")
print(f"  Posterior mean: {estimator.get_posterior_mean()*100:.4f}%")
print(f"  Posterior std: {estimator.get_posterior_std()*100:.4f}%")
print(f"  P(return > 0): {estimator.prob_positive_return()*100:.2f}%")
print(f"  5% VaR: {estimator.var_at_risk(0.05)*100:.4f}%")

## 4. Implement Prior and Posterior Updates

### Rolling Window Bayesian Update Strategy

We implement a strategy that:
1. Uses a rolling window of recent returns to continuously update our beliefs
2. Maintains posterior distributions that adapt to changing market conditions
3. Applies a decay factor to give more weight to recent observations

In [ ]:
class AdaptiveBayesianEstimator:
    """
    Adaptive Bayesian estimator with exponential decay for non-stationary returns.
    
    Uses a rolling window approach where older observations have less influence,
    allowing the model to adapt to changing market regimes.
    """
    
    def __init__(self, window_size=60, decay_factor=0.95):
        """
        Parameters:
        -----------
        window_size : int
            Number of recent observations to consider
        decay_factor : float
            Exponential decay for weighting observations (0 < decay < 1)
        """
        self.window_size = window_size
        self.decay_factor = decay_factor
        self.returns_buffer = []
        
        # Prior parameters (weakly informative)
        self.mu_0 = 0.0
        self.kappa_0 = 0.1  # Weak prior on mean
        self.alpha_0 = 2.0
        self.beta_0 = 0.0001
        
    def update(self, new_return):
        """Add new return observation."""
        self.returns_buffer.append(new_return)
        if len(self.returns_buffer) > self.window_size:
            self.returns_buffer.pop(0)
            
    def get_weighted_posterior(self):
        """
        Compute posterior using exponentially weighted observations.
        
        Returns:
        --------
        tuple: (mu_n, sigma_n, n_effective)
        """
        if len(self.returns_buffer) < 5:
            return self.mu_0, np.sqrt(self.beta_0 / (self.alpha_0 - 1)), 0
        
        returns = np.array(self.returns_buffer)
        n = len(returns)
        
        # Create exponential weights
        weights = np.array([self.decay_factor ** (n - 1 - i) for i in range(n)])
        weights = weights / weights.sum()  # Normalize
        
        # Weighted statistics
        weighted_mean = np.average(returns, weights=weights)
        weighted_var = np.average((returns - weighted_mean) ** 2, weights=weights)
        
        # Effective sample size
        n_eff = 1 / np.sum(weights ** 2)
        
        # Bayesian posterior with weighted data
        kappa_n = self.kappa_0 + n_eff
        mu_n = (self.kappa_0 * self.mu_0 + n_eff * weighted_mean) / kappa_n
        alpha_n = self.alpha_0 + n_eff / 2
        beta_n = self.beta_0 + 0.5 * n_eff * weighted_var + \
                 (self.kappa_0 * n_eff * (weighted_mean - self.mu_0) ** 2) / (2 * kappa_n)
        
        sigma_n = np.sqrt(beta_n / (alpha_n - 1)) if alpha_n > 1 else np.sqrt(weighted_var)
        
        return mu_n, sigma_n, n_eff
    
    def prob_positive_return(self):
        """Calculate P(next return > 0)."""
        mu_n, sigma_n, n_eff = self.get_weighted_posterior()
        
        if n_eff < 2:
            return 0.5  # Uncertain
        
        # Student-t predictive distribution
        df = max(2 * (self.alpha_0 + n_eff / 2), 3)
        scale = sigma_n * np.sqrt(1 + 1 / max(n_eff, 1))
        
        return 1 - stats.t.cdf(0, df=df, loc=mu_n, scale=scale)
    
    def get_sharpe_estimate(self, annualization=252):
        """Estimate Sharpe ratio from posterior."""
        mu_n, sigma_n, _ = self.get_weighted_posterior()
        if sigma_n > 0:
            return (mu_n * annualization) / (sigma_n * np.sqrt(annualization))
        return 0
    
    def reset(self):
        """Clear buffer."""
        self.returns_buffer = []


# Demonstrate the adaptive estimator on our data
print("Testing AdaptiveBayesianEstimator with rolling updates...\n")

adaptive_est = AdaptiveBayesianEstimator(window_size=60, decay_factor=0.95)

# Track posterior evolution
posterior_means = []
posterior_stds = []
prob_positive = []

for i, ret in enumerate(data['Returns'].values[:200]):
    adaptive_est.update(ret)
    mu, sigma, n_eff = adaptive_est.get_weighted_posterior()
    posterior_means.append(mu)
    posterior_stds.append(sigma)
    prob_positive.append(adaptive_est.prob_positive_return())

# Plot posterior evolution
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Posterior mean
axes[0].plot(posterior_means, color='blue', linewidth=1.5)
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0].fill_between(range(len(posterior_means)), 
                      np.array(posterior_means) - 2*np.array(posterior_stds),
                      np.array(posterior_means) + 2*np.array(posterior_stds),
                      alpha=0.3, color='blue', label='95% CI')
axes[0].set_ylabel('Posterior Mean Return')
axes[0].set_title('Evolution of Posterior Distribution')
axes[0].legend()

# Posterior std
axes[1].plot(posterior_stds, color='orange', linewidth=1.5)
axes[1].set_ylabel('Posterior Std')
axes[1].set_title('Uncertainty in Return Estimate')

# Probability of positive return
axes[2].plot(prob_positive, color='green', linewidth=1.5)
axes[2].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Neutral')
axes[2].set_ylabel('P(Return > 0)')
axes[2].set_xlabel('Trading Day')
axes[2].set_title('Probability of Positive Return')
axes[2].legend()

plt.tight_layout()
plt.show()

## 5. Generate Trading Signals

### Bayesian Signal Generation Rules

Our trading signals are based on posterior probabilities:

1. **Long Signal**: $P(r > 0 | D) > \theta_{long}$ (default: 0.55)
2. **Short Signal**: $P(r > 0 | D) < \theta_{short}$ (default: 0.45)
3. **Neutral**: Otherwise

We also incorporate:
- **Kelly Criterion** for position sizing based on posterior beliefs
- **Volatility scaling** to manage risk
- **Regime-based adjustments**

In [ ]:
class BayesianSignalGenerator:
    """
    Generate trading signals based on Bayesian posterior beliefs.
    """
    
    def __init__(self, 
                 long_threshold=0.55,
                 short_threshold=0.45,
                 window_size=60,
                 decay_factor=0.95,
                 use_kelly=True,
                 max_leverage=1.0,
                 vol_target=0.15):
        """
        Parameters:
        -----------
        long_threshold : float
            P(r>0) threshold to go long
        short_threshold : float
            P(r>0) threshold to go short (below this)
        window_size : int
            Lookback window for Bayesian estimation
        decay_factor : float
            Exponential decay weight
        use_kelly : bool
            Use Kelly criterion for position sizing
        max_leverage : float
            Maximum position size (1.0 = 100% invested)
        vol_target : float
            Target annualized volatility for position sizing
        """
        self.long_threshold = long_threshold
        self.short_threshold = short_threshold
        self.window_size = window_size
        self.decay_factor = decay_factor
        self.use_kelly = use_kelly
        self.max_leverage = max_leverage
        self.vol_target = vol_target
        
        self.estimator = AdaptiveBayesianEstimator(
            window_size=window_size, 
            decay_factor=decay_factor
        )
        
    def compute_kelly_fraction(self, prob_win, win_loss_ratio=1.0):
        """
        Compute Kelly criterion position size.
        
        Kelly Fraction = p - (1-p)/b
        where p = probability of winning, b = win/loss ratio
        """
        if prob_win <= 0 or prob_win >= 1:
            return 0
        kelly = prob_win - (1 - prob_win) / win_loss_ratio
        return np.clip(kelly, -self.max_leverage, self.max_leverage)
    
    def compute_vol_scaled_position(self, current_vol, annualization=252):
        """Scale position to target volatility."""
        if current_vol <= 0:
            return 1.0
        annualized_vol = current_vol * np.sqrt(annualization)
        return np.clip(self.vol_target / annualized_vol, 0, self.max_leverage)
    
    def generate_signal(self, returns_history, current_vol=None):
        """
        Generate trading signal based on posterior beliefs.
        
        Parameters:
        -----------
        returns_history : array-like
            Historical returns up to current time
        current_vol : float, optional
            Current realized volatility
            
        Returns:
        --------
        dict: Signal information including position size and metadata
        """
        # Reset and update estimator
        self.estimator.reset()
        for ret in returns_history[-self.window_size:]:
            self.estimator.update(ret)
        
        # Get posterior statistics
        prob_pos = self.estimator.prob_positive_return()
        mu, sigma, n_eff = self.estimator.get_weighted_posterior()
        sharpe_est = self.estimator.get_sharpe_estimate()
        
        # Determine direction
        if prob_pos > self.long_threshold:
            direction = 1  # Long
        elif prob_pos < self.short_threshold:
            direction = -1  # Short
        else:
            direction = 0  # Neutral
        
        # Position sizing
        if direction != 0:
            if self.use_kelly:
                prob_win = prob_pos if direction == 1 else (1 - prob_pos)
                base_size = abs(self.compute_kelly_fraction(prob_win))
            else:
                base_size = 1.0
            
            # Apply volatility scaling
            if current_vol is not None and current_vol > 0:
                vol_scale = self.compute_vol_scaled_position(current_vol)
                position_size = min(base_size * vol_scale, self.max_leverage)
            else:
                position_size = min(base_size, self.max_leverage)
        else:
            position_size = 0.0
        
        return {
            'direction': direction,
            'position_size': direction * position_size,
            'prob_positive': prob_pos,
            'posterior_mean': mu,
            'posterior_std': sigma,
            'sharpe_estimate': sharpe_est,
            'n_effective': n_eff
        }


# Test signal generation
signal_gen = BayesianSignalGenerator(
    long_threshold=0.55,
    short_threshold=0.45,
    window_size=60,
    use_kelly=True,
    max_leverage=1.0
)

# Generate signals over the dataset
signals_list = []
for i in range(60, len(data)):
    returns_history = data['Returns'].values[:i]
    current_vol = data['Volatility_20'].values[i-1] / np.sqrt(252) if i > 0 else None
    signal = signal_gen.generate_signal(returns_history, current_vol)
    signal['date'] = data.index[i]
    signals_list.append(signal)

signals_df = pd.DataFrame(signals_list)
signals_df.set_index('date', inplace=True)

# Display signal statistics
print("Signal Distribution:")
print(signals_df['direction'].value_counts())
print(f"\nMean P(positive): {signals_df['prob_positive'].mean():.4f}")
print(f"Mean position size: {signals_df['position_size'].abs().mean():.4f}")

# Plot signal distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# P(positive) histogram
axes[0, 0].hist(signals_df['prob_positive'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=0.55, color='green', linestyle='--', label='Long threshold')
axes[0, 0].axvline(x=0.45, color='red', linestyle='--', label='Short threshold')
axes[0, 0].set_xlabel('P(Return > 0)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Posterior Probability')
axes[0, 0].legend()

# Position sizes
axes[0, 1].hist(signals_df['position_size'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Position Size')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Position Sizes')

# Time series of positions
axes[1, 0].plot(signals_df.index, signals_df['position_size'], linewidth=0.8)
axes[1, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Position Size')
axes[1, 0].set_title('Position Size Over Time')

# Sharpe estimate evolution
axes[1, 1].plot(signals_df.index, signals_df['sharpe_estimate'], linewidth=0.8, color='purple')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('Date')
axes[1, 1].set_ylabel('Estimated Sharpe Ratio')
axes[1, 1].set_title('Rolling Sharpe Ratio Estimate')

plt.tight_layout()
plt.show()

## 6. Build Backtest Framework

Our backtesting engine simulates realistic trading including:
- **Transaction costs**: Slippage and commissions
- **Position tracking**: Long, short, and neutral positions
- **Portfolio value**: Mark-to-market tracking
- **Trade logging**: Detailed record of all trades

In [ ]:
class BayesianBacktester:
    """
    Comprehensive backtesting framework for Bayesian trading strategies.
    """
    
    def __init__(self,
                 initial_capital=100000,
                 transaction_cost_bps=5,
                 slippage_bps=2,
                 risk_free_rate=0.02):
        """
        Parameters:
        -----------
        initial_capital : float
            Starting portfolio value
        transaction_cost_bps : float
            Transaction costs in basis points
        slippage_bps : float
            Slippage estimate in basis points
        risk_free_rate : float
            Annual risk-free rate for Sharpe calculation
        """
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost_bps / 10000
        self.slippage = slippage_bps / 10000
        self.risk_free_rate = risk_free_rate
        
        # Results storage
        self.portfolio_values = []
        self.positions = []
        self.trades = []
        self.daily_returns = []
        
    def run_backtest(self, prices, signals_df, benchmark_returns=None):
        """
        Execute backtest simulation.
        
        Parameters:
        -----------
        prices : pd.Series
            Asset prices indexed by date
        signals_df : pd.DataFrame
            DataFrame with 'position_size' column indexed by date
        benchmark_returns : pd.Series, optional
            Benchmark returns for comparison
            
        Returns:
        --------
        dict: Backtest results including performance metrics
        """
        # Align data
        common_dates = prices.index.intersection(signals_df.index)
        prices = prices.loc[common_dates]
        signals_df = signals_df.loc[common_dates]
        
        # Initialize tracking
        portfolio_value = self.initial_capital
        current_position = 0
        shares_held = 0
        cash = self.initial_capital
        
        self.portfolio_values = [portfolio_value]
        self.positions = [0]
        self.trades = []
        
        for i in range(len(prices) - 1):
            date = prices.index[i]
            next_date = prices.index[i + 1]
            current_price = prices.iloc[i]
            next_price = prices.iloc[i + 1]
            
            target_position = signals_df['position_size'].iloc[i]
            
            # Calculate position change
            current_exposure = shares_held * current_price
            target_exposure = target_position * portfolio_value
            exposure_change = target_exposure - current_exposure
            
            # Execute trade if needed
            if abs(exposure_change) > 100:  # Minimum trade size
                # Calculate transaction costs
                trade_value = abs(exposure_change)
                total_cost = trade_value * (self.transaction_cost + self.slippage)
                
                # Update shares
                shares_change = exposure_change / current_price
                shares_held += shares_change
                cash -= exposure_change + total_cost
                
                self.trades.append({
                    'date': date,
                    'type': 'BUY' if exposure_change > 0 else 'SELL',
                    'shares': abs(shares_change),
                    'price': current_price,
                    'value': abs(exposure_change),
                    'cost': total_cost
                })
            
            # Calculate new portfolio value
            portfolio_value = cash + shares_held * next_price
            self.portfolio_values.append(portfolio_value)
            self.positions.append(shares_held * next_price / portfolio_value if portfolio_value > 0 else 0)
        
        # Create results DataFrame
        results_dates = prices.index[1:]
        self.results_df = pd.DataFrame({
            'date': results_dates,
            'portfolio_value': self.portfolio_values[1:],
            'position': self.positions[1:]
        }).set_index('date')
        
        self.results_df['strategy_returns'] = self.results_df['portfolio_value'].pct_change()
        self.results_df['cumulative_returns'] = (1 + self.results_df['strategy_returns'].fillna(0)).cumprod() - 1
        
        # Add benchmark if provided
        if benchmark_returns is not None:
            aligned_benchmark = benchmark_returns.loc[self.results_df.index]
            self.results_df['benchmark_returns'] = aligned_benchmark
            self.results_df['benchmark_cumulative'] = (1 + aligned_benchmark.fillna(0)).cumprod() - 1
        
        return self.calculate_metrics()
    
    def calculate_metrics(self):
        """Calculate comprehensive performance metrics."""
        returns = self.results_df['strategy_returns'].dropna()
        
        if len(returns) < 2:
            return {}
        
        # Basic metrics
        total_return = (self.results_df['portfolio_value'].iloc[-1] / self.initial_capital) - 1
        ann_return = (1 + total_return) ** (252 / len(returns)) - 1
        ann_volatility = returns.std() * np.sqrt(252)
        
        # Risk-adjusted metrics
        excess_returns = returns - self.risk_free_rate / 252
        sharpe_ratio = excess_returns.mean() / returns.std() * np.sqrt(252) if returns.std() > 0 else 0
        
        # Sortino Ratio (downside deviation)
        downside_returns = returns[returns < 0]
        downside_std = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 1
        sortino_ratio = ann_return / downside_std if downside_std > 0 else 0
        
        # Maximum Drawdown
        cumulative = (1 + returns).cumprod()
        rolling_max = cumulative.expanding().max()
        drawdowns = cumulative / rolling_max - 1
        max_drawdown = drawdowns.min()
        
        # Calmar Ratio
        calmar_ratio = ann_return / abs(max_drawdown) if max_drawdown != 0 else 0
        
        # Win/Loss statistics
        winning_days = (returns > 0).sum()
        losing_days = (returns < 0).sum()
        win_rate = winning_days / (winning_days + losing_days) if (winning_days + losing_days) > 0 else 0
        
        avg_win = returns[returns > 0].mean() if len(returns[returns > 0]) > 0 else 0
        avg_loss = abs(returns[returns < 0].mean()) if len(returns[returns < 0]) > 0 else 1
        profit_factor = (avg_win * winning_days) / (avg_loss * losing_days) if losing_days > 0 else np.inf
        
        # Trade statistics
        n_trades = len(self.trades)
        total_costs = sum(t['cost'] for t in self.trades)
        
        metrics = {
            'Total Return': f"{total_return*100:.2f}%",
            'Annualized Return': f"{ann_return*100:.2f}%",
            'Annualized Volatility': f"{ann_volatility*100:.2f}%",
            'Sharpe Ratio': f"{sharpe_ratio:.3f}",
            'Sortino Ratio': f"{sortino_ratio:.3f}",
            'Max Drawdown': f"{max_drawdown*100:.2f}%",
            'Calmar Ratio': f"{calmar_ratio:.3f}",
            'Win Rate': f"{win_rate*100:.2f}%",
            'Profit Factor': f"{profit_factor:.3f}",
            'Number of Trades': n_trades,
            'Total Transaction Costs': f"${total_costs:,.2f}",
            'Final Portfolio Value': f"${self.results_df['portfolio_value'].iloc[-1]:,.2f}"
        }
        
        # Store numeric values for comparison
        self.numeric_metrics = {
            'total_return': total_return,
            'ann_return': ann_return,
            'ann_volatility': ann_volatility,
            'sharpe_ratio': sharpe_ratio,
            'sortino_ratio': sortino_ratio,
            'max_drawdown': max_drawdown,
            'calmar_ratio': calmar_ratio,
            'win_rate': win_rate,
            'profit_factor': profit_factor
        }
        
        return metrics


# Run the backtest
print("Running Bayesian Strategy Backtest...")
print("="*50)

backtester = BayesianBacktester(
    initial_capital=100000,
    transaction_cost_bps=5,
    slippage_bps=2
)

# Run backtest
metrics = backtester.run_backtest(
    prices=data['Adj Close'],
    signals_df=signals_df,
    benchmark_returns=data['Returns']
)

# Display metrics
print("\nBacktest Results:")
print("-"*50)
for metric, value in metrics.items():
    print(f"{metric:30s}: {value}")
    
print(f"\nBacktest Period: {signals_df.index[0].date()} to {signals_df.index[-1].date()}")
print(f"Total Trading Days: {len(signals_df)}")

## 7. Calculate Performance Metrics

Let's compute additional metrics and compare our Bayesian strategy against a simple buy-and-hold benchmark.

In [ ]:
def calculate_rolling_metrics(returns, window=252):
    """Calculate rolling performance metrics."""
    rolling_return = returns.rolling(window=window).mean() * 252
    rolling_vol = returns.rolling(window=window).std() * np.sqrt(252)
    rolling_sharpe = rolling_return / rolling_vol
    
    # Rolling max drawdown
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.rolling(window=window, min_periods=1).max()
    rolling_dd = cumulative / rolling_max - 1
    rolling_max_dd = rolling_dd.rolling(window=window, min_periods=1).min()
    
    return rolling_return, rolling_vol, rolling_sharpe, rolling_max_dd


def calculate_benchmark_metrics(returns, risk_free_rate=0.02):
    """Calculate buy-and-hold benchmark metrics."""
    total_return = (1 + returns).prod() - 1
    ann_return = (1 + total_return) ** (252 / len(returns)) - 1
    ann_vol = returns.std() * np.sqrt(252)
    sharpe = (ann_return - risk_free_rate) / ann_vol
    
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.expanding().max()
    max_dd = (cumulative / rolling_max - 1).min()
    
    return {
        'Total Return': total_return,
        'Annualized Return': ann_return,
        'Annualized Volatility': ann_vol,
        'Sharpe Ratio': sharpe,
        'Max Drawdown': max_dd
    }


# Calculate benchmark metrics
benchmark_returns = data.loc[backtester.results_df.index, 'Returns']
benchmark_metrics = calculate_benchmark_metrics(benchmark_returns)

# Compare Strategy vs Benchmark
print("=" * 60)
print("STRATEGY vs BENCHMARK COMPARISON")
print("=" * 60)
print(f"\n{'Metric':<25} {'Strategy':>15} {'Benchmark':>15}")
print("-" * 60)

comparison_metrics = [
    ('Total Return', backtester.numeric_metrics['total_return'], benchmark_metrics['Total Return']),
    ('Ann. Return', backtester.numeric_metrics['ann_return'], benchmark_metrics['Annualized Return']),
    ('Ann. Volatility', backtester.numeric_metrics['ann_volatility'], benchmark_metrics['Annualized Volatility']),
    ('Sharpe Ratio', backtester.numeric_metrics['sharpe_ratio'], benchmark_metrics['Sharpe Ratio']),
    ('Max Drawdown', backtester.numeric_metrics['max_drawdown'], benchmark_metrics['Max Drawdown']),
]

for name, strat_val, bench_val in comparison_metrics:
    if 'Drawdown' in name:
        print(f"{name:<25} {strat_val*100:>14.2f}% {bench_val*100:>14.2f}%")
    elif 'Ratio' in name:
        print(f"{name:<25} {strat_val:>15.3f} {bench_val:>15.3f}")
    else:
        print(f"{name:<25} {strat_val*100:>14.2f}% {bench_val*100:>14.2f}%")

# Calculate information ratio
strategy_returns = backtester.results_df['strategy_returns'].dropna()
tracking_error = (strategy_returns - benchmark_returns.loc[strategy_returns.index]).std() * np.sqrt(252)
excess_return = backtester.numeric_metrics['ann_return'] - benchmark_metrics['Annualized Return']
information_ratio = excess_return / tracking_error if tracking_error > 0 else 0

print(f"\n{'Information Ratio':<25} {information_ratio:>15.3f}")
print(f"{'Tracking Error':<25} {tracking_error*100:>14.2f}%")

# Monthly returns analysis
monthly_returns = strategy_returns.resample('M').apply(lambda x: (1+x).prod()-1)
benchmark_monthly = benchmark_returns.resample('M').apply(lambda x: (1+x).prod()-1)

print(f"\n{'Monthly Analysis':<25}")
print(f"{'Best Month':<25} {monthly_returns.max()*100:>14.2f}% {benchmark_monthly.max()*100:>14.2f}%")
print(f"{'Worst Month':<25} {monthly_returns.min()*100:>14.2f}% {benchmark_monthly.min()*100:>14.2f}%")
print(f"{'% Positive Months':<25} {(monthly_returns>0).mean()*100:>14.2f}% {(benchmark_monthly>0).mean()*100:>14.2f}%")

## 8. Visualize Strategy Results

Comprehensive visualization of the backtest results including equity curves, drawdowns, and return distributions.

In [ ]:
def plot_backtest_results(results_df, benchmark_returns, title="Bayesian Trading Strategy"):
    """Create comprehensive visualization of backtest results."""
    
    fig = plt.figure(figsize=(16, 14))
    
    # 1. Equity Curve
    ax1 = plt.subplot(3, 2, 1)
    strategy_equity = results_df['portfolio_value'] / results_df['portfolio_value'].iloc[0]
    benchmark_equity = (1 + benchmark_returns.loc[results_df.index].fillna(0)).cumprod()
    
    ax1.plot(strategy_equity.index, strategy_equity.values, label='Bayesian Strategy', linewidth=2)
    ax1.plot(benchmark_equity.index, benchmark_equity.values, label='Buy & Hold', linewidth=2, alpha=0.7)
    ax1.set_ylabel('Portfolio Value (Normalized)')
    ax1.set_title('Equity Curve Comparison')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # 2. Drawdown Chart
    ax2 = plt.subplot(3, 2, 2)
    strategy_returns = results_df['strategy_returns'].dropna()
    
    # Strategy drawdown
    strategy_cumulative = (1 + strategy_returns).cumprod()
    strategy_rolling_max = strategy_cumulative.expanding().max()
    strategy_dd = strategy_cumulative / strategy_rolling_max - 1
    
    # Benchmark drawdown
    benchmark_cumulative = (1 + benchmark_returns.loc[results_df.index].dropna()).cumprod()
    benchmark_rolling_max = benchmark_cumulative.expanding().max()
    benchmark_dd = benchmark_cumulative / benchmark_rolling_max - 1
    
    ax2.fill_between(strategy_dd.index, strategy_dd.values, 0, alpha=0.3, label='Strategy DD')
    ax2.fill_between(benchmark_dd.index, benchmark_dd.values, 0, alpha=0.3, label='Benchmark DD')
    ax2.set_ylabel('Drawdown')
    ax2.set_title('Drawdown Comparison')
    ax2.legend(loc='lower left')
    ax2.grid(True, alpha=0.3)
    
    # 3. Return Distribution
    ax3 = plt.subplot(3, 2, 3)
    ax3.hist(strategy_returns * 100, bins=50, alpha=0.6, label='Strategy', density=True)
    ax3.hist(benchmark_returns.loc[results_df.index] * 100, bins=50, alpha=0.6, label='Benchmark', density=True)
    ax3.axvline(x=0, color='black', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Daily Return (%)')
    ax3.set_ylabel('Density')
    ax3.set_title('Return Distribution')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Rolling Sharpe Ratio
    ax4 = plt.subplot(3, 2, 4)
    window = 63  # Quarterly
    rolling_sharpe_strat = (strategy_returns.rolling(window).mean() / 
                            strategy_returns.rolling(window).std()) * np.sqrt(252)
    rolling_sharpe_bench = (benchmark_returns.loc[results_df.index].rolling(window).mean() / 
                            benchmark_returns.loc[results_df.index].rolling(window).std()) * np.sqrt(252)
    
    ax4.plot(rolling_sharpe_strat.index, rolling_sharpe_strat.values, label='Strategy', linewidth=1.5)
    ax4.plot(rolling_sharpe_bench.index, rolling_sharpe_bench.values, label='Benchmark', linewidth=1.5, alpha=0.7)
    ax4.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax4.set_ylabel('Sharpe Ratio')
    ax4.set_title(f'Rolling Sharpe Ratio ({window}-day)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # 5. Position Over Time
    ax5 = plt.subplot(3, 2, 5)
    ax5.fill_between(results_df.index, results_df['position'].values, 0, 
                     where=results_df['position'] > 0, alpha=0.5, color='green', label='Long')
    ax5.fill_between(results_df.index, results_df['position'].values, 0,
                     where=results_df['position'] < 0, alpha=0.5, color='red', label='Short')
    ax5.set_ylabel('Position')
    ax5.set_xlabel('Date')
    ax5.set_title('Position Exposure Over Time')
    ax5.legend()
    ax5.grid(True, alpha=0.3)
    
    # 6. Monthly Returns Heatmap
    ax6 = plt.subplot(3, 2, 6)
    monthly = strategy_returns.resample('M').apply(lambda x: (1+x).prod()-1) * 100
    monthly_df = pd.DataFrame({
        'Year': monthly.index.year,
        'Month': monthly.index.month,
        'Return': monthly.values
    })
    monthly_pivot = monthly_df.pivot_table(values='Return', index='Year', columns='Month', aggfunc='first')
    
    sns.heatmap(monthly_pivot, annot=True, fmt='.1f', cmap='RdYlGn', center=0,
                ax=ax6, cbar_kws={'label': 'Monthly Return (%)'})
    ax6.set_title('Monthly Returns Heatmap')
    ax6.set_xlabel('Month')
    ax6.set_ylabel('Year')
    
    plt.suptitle(title, fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()


# Generate the comprehensive visualization
plot_backtest_results(
    backtester.results_df,
    data['Returns'],
    title=f"Bayesian Trading Strategy - {TICKER}"
)

## 9. Sensitivity Analysis

Let's analyze how the strategy performs with different parameter settings.

In [ ]:
def run_parameter_sweep(data, window_sizes, thresholds):
    """
    Run parameter sweep to find optimal strategy parameters.
    
    Parameters:
    -----------
    data : pd.DataFrame
        Market data
    window_sizes : list
        List of window sizes to test
    thresholds : list
        List of threshold values (for long_threshold, short = 1 - threshold)
        
    Returns:
    --------
    pd.DataFrame: Results for each parameter combination
    """
    results = []
    
    total_combinations = len(window_sizes) * len(thresholds)
    print(f"Testing {total_combinations} parameter combinations...")
    
    for window in window_sizes:
        for thresh in thresholds:
            # Generate signals
            signal_gen = BayesianSignalGenerator(
                long_threshold=thresh,
                short_threshold=1-thresh,
                window_size=window,
                use_kelly=True,
                max_leverage=1.0
            )
            
            signals_list = []
            for i in range(window, len(data)):
                returns_history = data['Returns'].values[:i]
                current_vol = data['Volatility_20'].values[i-1] / np.sqrt(252)
                signal = signal_gen.generate_signal(returns_history, current_vol)
                signal['date'] = data.index[i]
                signals_list.append(signal)
            
            signals_df = pd.DataFrame(signals_list).set_index('date')
            
            # Run backtest
            backtester = BayesianBacktester(initial_capital=100000)
            metrics = backtester.run_backtest(
                prices=data['Adj Close'],
                signals_df=signals_df
            )
            
            results.append({
                'window_size': window,
                'threshold': thresh,
                'sharpe_ratio': backtester.numeric_metrics['sharpe_ratio'],
                'total_return': backtester.numeric_metrics['total_return'],
                'max_drawdown': backtester.numeric_metrics['max_drawdown'],
                'win_rate': backtester.numeric_metrics['win_rate'],
                'calmar_ratio': backtester.numeric_metrics['calmar_ratio']
            })
    
    return pd.DataFrame(results)


# Run parameter sweep
window_sizes = [30, 45, 60, 90, 120]
thresholds = [0.52, 0.55, 0.58, 0.60, 0.62]

sweep_results = run_parameter_sweep(data, window_sizes, thresholds)

# Find best parameters
best_sharpe_idx = sweep_results['sharpe_ratio'].idxmax()
best_params = sweep_results.loc[best_sharpe_idx]

print("\n" + "="*60)
print("PARAMETER SWEEP RESULTS")
print("="*60)
print(f"\nBest Parameters (by Sharpe Ratio):")
print(f"  Window Size: {int(best_params['window_size'])}")
print(f"  Threshold: {best_params['threshold']:.2f}")
print(f"  Sharpe Ratio: {best_params['sharpe_ratio']:.3f}")
print(f"  Total Return: {best_params['total_return']*100:.2f}%")
print(f"  Max Drawdown: {best_params['max_drawdown']*100:.2f}%")

# Visualize parameter sensitivity
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Heatmap of Sharpe Ratios
pivot_sharpe = sweep_results.pivot_table(values='sharpe_ratio', 
                                          index='window_size', 
                                          columns='threshold')
sns.heatmap(pivot_sharpe, annot=True, fmt='.3f', cmap='RdYlGn', ax=axes[0])
axes[0].set_title('Sharpe Ratio')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Window Size')

# Heatmap of Total Returns
pivot_return = sweep_results.pivot_table(values='total_return', 
                                          index='window_size', 
                                          columns='threshold') * 100
sns.heatmap(pivot_return, annot=True, fmt='.1f', cmap='RdYlGn', ax=axes[1])
axes[1].set_title('Total Return (%)')
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('Window Size')

# Heatmap of Max Drawdown
pivot_dd = sweep_results.pivot_table(values='max_drawdown', 
                                      index='window_size', 
                                      columns='threshold') * 100
sns.heatmap(pivot_dd, annot=True, fmt='.1f', cmap='RdYlGn_r', ax=axes[2])
axes[2].set_title('Max Drawdown (%)')
axes[2].set_xlabel('Threshold')
axes[2].set_ylabel('Window Size')

plt.suptitle('Parameter Sensitivity Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Key Takeaways & Next Steps

### Summary

This notebook demonstrated a complete **Bayesian trading strategy** framework:

1. **Normal-Inverse-Gamma Conjugate Priors**: Enable efficient online updates of return distribution beliefs
2. **Adaptive Estimation**: Exponential decay weighting adapts to changing market regimes
3. **Probabilistic Signals**: Trading decisions based on $P(\text{return} > 0)$ rather than point estimates
4. **Kelly Criterion Position Sizing**: Optimal bet sizing based on posterior beliefs
5. **Comprehensive Backtesting**: Realistic simulation with transaction costs

### Key Bayesian Advantages

| Aspect | Frequentist Approach | Bayesian Approach |
|--------|---------------------|-------------------|
| Uncertainty | Point estimates | Full posterior distributions |
| Updates | Re-estimate from scratch | Efficient conjugate updates |
| Small samples | Unreliable | Prior regularizes estimates |
| Decision making | Ad-hoc thresholds | Principled probabilistic |

### Extensions to Explore

1. **Bayesian Model Comparison**: Use Bayes factors to select between different model structures
2. **Hierarchical Models**: Pool information across multiple assets
3. **MCMC Sampling**: For more complex non-conjugate models (PyMC, Stan)
4. **Regime-Switching**: Hidden Markov Models with Bayesian inference
5. **Bayesian Neural Networks**: Uncertainty quantification in deep learning

### References

- Gelman et al. "Bayesian Data Analysis" (3rd Edition)
- Murphy, K. "Machine Learning: A Probabilistic Perspective"
- López de Prado, "Advances in Financial Machine Learning"

In [ ]:
# Final Summary Statistics
print("="*60)
print("FINAL BACKTEST SUMMARY")
print("="*60)
print(f"\nStrategy: Bayesian Posterior-Based Trading")
print(f"Asset: {TICKER}")
print(f"Period: {START_DATE} to {END_DATE}")
print(f"\nBayesian Model: Normal-Inverse-Gamma Conjugate Prior")
print(f"Signal Rule: Long when P(return > 0) > {signal_gen.long_threshold}")
print(f"            Short when P(return > 0) < {signal_gen.short_threshold}")
print(f"Position Sizing: Kelly Criterion with volatility scaling")
print(f"\n" + "-"*60)
print("Performance Metrics:")
for metric, value in metrics.items():
    print(f"  {metric}: {value}")
print("="*60)